## 迭代器

**迭代器提供 next 方法。调用这个方法，可以得到这个容器的下一个对象或者 StopIteration 错误。比如，字典采用哈希表实现，那么只需要知道，next 函数可以不重复不遗漏地一个一个拿到所有元素即可。**

可迭代对象，通过 iter() 函数返回一个迭代器，再通过 next() 函数就可以实现遍历。for in 语句将这个过程隐式化

**判断是否可以迭代**

In [1]:
def is_itreable(param):
    try:
        iter(param)
        return True
    except TypeError:
        return False
params=[1234,'1234',[1,2,3,4],set([1,2,3,4]),{1:1,2:2,3:3},(1,2,3,4)]
for param in params:
    print(f"{param} is iterable? {is_itreable(param)}")

1234 is iterable? False
1234 is iterable? True
[1, 2, 3, 4] is iterable? True
{1, 2, 3, 4} is iterable? True
{1: 1, 2: 2, 3: 3} is iterable? True
(1, 2, 3, 4) is iterable? True


## 生成器
（懒人版本的迭代器）

在迭代器中，我们想要枚举它的元素，需要提前生成这些元素。<br/>样例：

import os
import psutil
#显示当前python程序占用的内存大小
def show_memory_info(hint):
    pid=os.getpid()
    p=psutil.Process(pid)
    info=p.memory_full_info()
    print('{} memory used: {} MB'.format(hint, memory))
def test_iterator():
    show_memory_info("initing iterator")
    list1=[i for i in range(100000000)]
    show_memory_info("after iterator initiated")
    print(sum(list1))
    show_memory_info("after sum called")
def test_generator():
    show_memory_info('initing generator')
    list_2 = (i for i in range(100000000))
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')
%time test_iterator()
%time test_generator()

当我们只需要使用每个元素的信息，而不是需要整体的信息时，可以使用生成器。在调用 next() 函数的时候，才会生成下一个变量。生成器在 Python 的写法是用小括号括起来，(i for i in range(100000000))，即初始化了一个生成器。

**生成器并不会像迭代器一样占用大量内存，只有在被使用的时候才会调用。**

**数学中有一个恒等式，(1 + 2 + 3 + ... + n)^2 = 1^3 + 2^3 + 3^3 + ... + n^3**

In [6]:
def generator(k):
    i = 1
    while True:
        yield i ** k    #程序暂停，返回next（）函数     yield 返回一个生成器
        i += 1           #1  2  3
                         #1  8  27
gen_1 = generator(1)  
gen_3 = generator(3)
print(gen_1)
print(gen_3)
 
def get_sum(n):
    sum_1, sum_3 = 0, 0
    for i in range(n):
        next_1 = next(gen_1) #从yield继续执行，局部变量i继续累加。
        next_3 = next(gen_3)
        print('next_1 = {}, next_3 = {}'.format(next_1, next_3))
        sum_1 += next_1   #1  3  6
        sum_3 += next_3   #1  8   27
    print(sum_1 * sum_1, sum_3)
 
get_sum(3)

<generator object generator at 0x000002765DAB68F0>
<generator object generator at 0x000002765DAB6650>
next_1 = 1, next_3 = 1
next_1 = 2, next_3 = 8
next_1 = 3, next_3 = 27
36 36


**给定一个 list 和一个指定数字，求这个数字在 list 中的位置。**


1.枚举法

In [12]:
def index_normal(L, target):
    result = []
    for i, num in enumerate(L):  #enumerate返回索引序列 [(0,"e1"),(1,"e2")]
        if num == target:
            result.append(i)
    return result
print(index_normal([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2))

[2, 5, 9]


2.迭代器法

In [13]:
def index_normal(L,target):
    for i,num in enumerate(L):
        if num==target:
            yield i  #程序暂停，局部变量仍然保留。
print(list(index_normal([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)))

[2, 5, 9]


**给定两个序列，判定第一个是不是第二个的子序列。<br/>比如[1,3,5]是[1,2,3,4,5]的子序列**

常规算法是贪心算法。我们维护两个指针指向两个列表的最开始，然后对第二个序列一路扫过去，如果某个数字和第一个指针指的一样，那么就把第一个指针前进一步。第一个指针移出第一个序列最后一个元素的时候，返回 True，否则返回 False。

**迭代器生成**

In [38]:
def is_subsequence(a, b):
    b = iter(b)
    return all(i in b for i in a)
print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

True
False


In [42]:
def is_subsequence(a, b):
    b = iter(b) #将b列表转换为一个迭代器，迭代器只能遍历一次，只能向前遍历，不能回退。
#     print(b)
#     gen = (i for i in a) #产生一个生成器遍历a
#     print(gen) 
#     for i in gen:
#         print(i)
#     gen = ((i in b) for i in a)
#     print(gen)              #(i in b)等价于 
#                             # while True:
#                             #val=next(b)  if val==i:  yield True !!bext()函数运行，保留当前的指针
#     for i in gen:
#         print(i)  #all() 函数⽤于判断给定的可迭代参数 iterable 中的所有元素是否都为 TRUE，如果是返回 True，否则返回 False。
#                 #元素除了是 0、空、None、False 外都算 True。
    return all((i in b for i in a))   #执行以上代码，抛出错误，是因为生成器只能遍历一次
print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))

True


In [47]:
#!!!next() 函数运行的时候，保存了当前的指针，所以生成器只能遍历一次，继续调用会抛出raise Stopiteration
b = (i for i in range(5))
 
print(2 in b)
print(4 in b)
print(4 in b)
print (all((2 in b) for i in [2]))

True
True
False
False


**all() 函数用来判断一个迭代器的元素是否全部为 True，如果是则返回 True，否则就返回 False**

next函数可能有些误区，迭代完成后继续调用next函数会返回默认值None。
 iterator.__next__() 方法和 next(iterator, default=None) 函数的区别在于：前者迭代完成后会抛出StopIteration错误，中断程序运行，而后者会返回一个默认值None（可以指定），不会报错和中断程序运行。